enter your name and have 50 photos taken

In [ ]:
import cv2
import os

def make_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

def take_pic(name, folder_name):
    cam = cv2.VideoCapture(0)
    make_folder(folder_name)

    for i in range(50):
        _, pic = cam.read()
        pic_name = f"{folder_name}/{name}_{i+1}.jpg"
        cv2.imwrite(pic_name, pic)
        print(f"{i+1}. pic taked: {pic_name}")

    cam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    name = input("enter your name: ")
    folder_name = f"datas/{name}"
    take_pic(name, folder_name)


model training

In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import os

def recognize(folder_name):
    tags = []
    faces = []
    
    for persons in os.listdir(folder_name):
        for folder in os.listdir(os.path.join(folder_name, persons)):
            pic_path = os.path.join(folder_name, persons, folder)
            img = cv2.imread(pic_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (100, 100))
            
            flattened_img = img.flatten()
            
            faces.append(flattened_img)
            tags.append(persons)

    return np.array(faces), np.array(tags)

def build_model():
    folder_name = "datas"
    X, y = recognize(folder_name)

    tag_encoder = LabelEncoder()
    y = tag_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = SVC(kernel='linear', probability=True)
    model.fit(X_train, y_train)

    return model, tag_encoder

if __name__ == "__main__":
    model, tag_encoder = build_model()
    print("model built.")


let's try the model

In [ ]:
import cv2
import numpy as np
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
def get_name(pic, name):
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(pic, name, (10, 30), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

if __name__ == "__main__":
    cam = cv2.VideoCapture(0)
    model, etiket_encoder = build_model()

    while True:
        _, pic = cam.read()
        gray = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_part = gray[y:y+h, x:x+w]
            face_part = cv2.resize(face_part, (100, 100))
            face_part = face_part.flatten()

            models_guess = model.predict([face_part])
            probability = np.max(model.predict_proba([face_part]))

            name = tag_encoder.inverse_transform(models_guess)[0]

            get_name(pic, name)

        cv2.imshow("face recognition", pic)

        # press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cam.release()
    cv2.destroyAllWindows()
